# Fit Fragments to BRD4 Ligands

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from rdkit import Chem

import open3d as o3d
import molgrid

from openbabel import pybel

import os
import tqdm

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
import sys
sys.path.append("..")
sys.path.append("../../ligan-EVOTEC")

from utils import show_molecule_idx, show_two_mols, transform_and_add_conformer, show_conformer_idx

from score_pcd import fit_and_score
from molgrid_diff import grid_diff
from fit_to_grid import molgrid_diff_to_mol

In [4]:
from molecules import rd_mol_to_ob_mol 

Load CDK2 inhibitors:

In [5]:
path = "ligands/CDK2"
files = [os.path.join(path, f) for f in os.listdir(path) if os.path.splitext(f)[-1] == ".pcd" and os.path.splitext(f)[0][-4:] == "tran"]

# Order ligands
# This should make the three chemical series pop-up in the PCD fit
names = {
    "4ek4_B_1CK": "CS1",
    "4ek5_B_03K": "CS3",
    "4fkg_B_4CK": "CS4",
    "4fki_B_09K": "CS9",
    "4fkj_B_11K": "CS11",
    "3sw4_B_18K": "CS18",
    "3sw7_B_19K": "CS19",
    "4fko_B_20K": "CS20",
    "4fkp_B_LS5": "CS241",
    "4fkq_B_42K": "CS242",
    "4fkr_B_45K": "CS245",
    "4fks_B_46K": "CS246",
    "4fkt_B_48K": "CS248",
    "4fku_D_60K": "CS260",
    "4fkv_B_61K": "CS261",
    "4fkw_B_62K": "CS262",
}

files.sort(key=lambda f: int(names[os.path.splitext(os.path.basename(f))[0].replace("_tran", "")].replace("CS", "")))

print(files)

pcds = []
mols = []
for f in files:
    pcd = o3d.io.read_point_cloud(f)
    pcds.append(pcd)

    s = Chem.SDMolSupplier(f.replace(".pcd", ".sdf"))
    mol = next(s)
    mols.append(mol)

['ligands/CDK2/4ek4_B_1CK_tran.pcd', 'ligands/CDK2/4ek5_B_03K_tran.pcd', 'ligands/CDK2/4fkg_B_4CK_tran.pcd', 'ligands/CDK2/4fki_B_09K_tran.pcd', 'ligands/CDK2/4fkj_B_11K_tran.pcd', 'ligands/CDK2/3sw4_B_18K_tran.pcd', 'ligands/CDK2/3sw7_B_19K_tran.pcd', 'ligands/CDK2/4fko_B_20K_tran.pcd', 'ligands/CDK2/4fkp_B_LS5_tran.pcd', 'ligands/CDK2/4fkq_B_42K_tran.pcd', 'ligands/CDK2/4fkr_B_45K_tran.pcd', 'ligands/CDK2/4fks_B_46K_tran.pcd', 'ligands/CDK2/4fkt_B_48K_tran.pcd', 'ligands/CDK2/4fku_D_60K_tran.pcd', 'ligands/CDK2/4fkv_B_61K_tran.pcd', 'ligands/CDK2/4fkw_B_62K_tran.pcd']


Load fragments from DSI Poised library:

In [6]:
fragpath=path = "../fragments/DSIpoised"
fragfiles = [os.path.join(fragpath, f) for f in os.listdir(fragpath) if os.path.splitext(f)[-1] == ".pcd"]

print(fragfiles[:5])

fragpcds = []
fragmols = []
for ff in tqdm.tqdm(fragfiles):
    pcd = o3d.io.read_point_cloud(ff)
    fragpcds.append(pcd)

    s = Chem.SDMolSupplier(ff.replace(".pcd", ".sdf"))
    mol = next(s)
    fragmols.append(mol)

['../fragments/DSIpoised/fragment_0.pcd', '../fragments/DSIpoised/fragment_100.pcd', '../fragments/DSIpoised/fragment_101.pcd', '../fragments/DSIpoised/fragment_102.pcd', '../fragments/DSIpoised/fragment_103.pcd']


100%|██████████| 860/860 [00:06<00:00, 131.23it/s]


In [7]:
molid = 8

show_molecule_idx(molid, mols)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Alignment and Fitting

In [8]:
alignments = []
for fidx, (fragpcd, fragmol) in tqdm.tqdm(enumerate(zip(fragpcds, fragmols)), total=len(fragpcds)):
    fit, cfit, tran = fit_and_score((fragpcd, pcds[molid]), voxel_size=0.5, threshold=0.5)
    alignments.append((fidx, cfit.fitness, tran))
    

100%|██████████| 860/860 [03:35<00:00,  3.99it/s]


In [9]:
minfitness = 0.85

goodalignments = [a for a in alignments if a[1] >= minfitness]
goodalignments.sort(key=lambda t: t[1], reverse=True)

In [10]:
[a[0:2] for a in goodalignments]

[(668, 0.9057017543859649),
 (756, 0.9002433090024331),
 (822, 0.8986486486486487),
 (804, 0.8926380368098159),
 (191, 0.8865740740740741),
 (5, 0.8822055137844611),
 (525, 0.8737864077669902),
 (454, 0.8732394366197183),
 (359, 0.8731988472622478),
 (749, 0.869757174392936),
 (475, 0.8677685950413223),
 (374, 0.8618925831202046),
 (551, 0.8617021276595744),
 (613, 0.8581907090464548),
 (157, 0.8574660633484162),
 (783, 0.8551859099804305),
 (516, 0.8535714285714285),
 (690, 0.852760736196319),
 (233, 0.8509485094850948),
 (729, 0.85)]

In [11]:
for (idx, fitness, tran) in goodalignments:
    transform_and_add_conformer(fragmols[idx], tran, fromConfId=0, toConfId=1)

In [12]:
fragid = goodalignments[0][0]
show_two_mols(mols[molid], fragmols[fragid], confId2=1)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [13]:
fragid = goodalignments[1][0]
show_two_mols(mols[molid], fragmols[fragid], confId2=1)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [14]:
fragid = goodalignments[2][0]
show_two_mols(mols[molid], fragmols[fragid], confId2=1)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Compute and Fit Density Difference

The aligned conformation is stored as a conformer of the original fragment, therefore one needs to ensure that the density corresponding to the correct conformer is removed.

In [15]:
ligmap = "../files/ligmap"
typer = molgrid.FileMappedGninaTyper(ligmap)

In [16]:
obmol = pybel.Molecule(rd_mol_to_ob_mol(mols[molid]))

rdfrag = fragmols[goodalignments[0][0]]
# Get correct conformer for rdfrag
# confId=1 corresponds to the aligned conformers
#rdfrag = Chem.MolFromMolBlock(Chem.MolToMolBlock(rdfrag, confId=1))
# https://github.com/rdkit/rdkit/discussions/4520
crdfrag = Chem.Mol(rdfrag, True) # Copy molecule
crdfrag.AddConformer(rdfrag.GetConformer(1), assignId=True) # Add conformer to copy of molecule

obfrag = pybel.Molecule(rd_mol_to_ob_mol(crdfrag)) # Best aligned fragment (confId=1)

In [17]:
diff, c = grid_diff(obmol, obfrag, 23.5, 0.5, typer)
rdmolfit = molgrid_diff_to_mol(diff, c, 0.5, ligmap, scaffold=None)

../../ligan-EVOTEC/fitting.py:115: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  values = torch.tensor(mgrid.values,device=device)


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
show_two_mols(mols[molid], rdmolfit)

In [ ]:
show_molecule_idx(0, [rdmolfit])

In [ ]:
rdmolfit = molgrid_diff_to_mol(diff, c, 0.5, ligmap, scaffold=rdfrag, verbose=True)